In [0]:
#Pyspark RandomForestClassifer
#Let's import the required libraries
import pyspark
from __future__ import print_function
from pyspark.sql import SparkSession
from pyspark.sql.functions import count,when,col
from pyspark.ml.feature import StandardScaler,StringIndexer,VectorAssembler,VectorIndexer
from pyspark.ml.classification import BinaryRandomForestClassificationSummary,BinaryRandomForestClassificationTrainingSummary
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

In [0]:
#Load the data in to the notebook
df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/praneethsanthosh555@gmail.com/winequality_red-2.csv",header=True)
df.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5|
 7.8| 0.88| 0.0| 2.6| 0.098| 25.0| 67.0| 0.9968| 3.2| 0.68| 9.8| 5|
 7.8| 0.76| 0.04| 2.3| 0.092| 15.0| 54.0| 0.997|3.26| 0.65| 9.8| 5|
 11.2| 0.28| 0.56| 1.9| 0.075| 17.0| 60.0| 0.998|3.16| 0.58| 9.8| 6|
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
only showing top 5 rows

In [0]:
df.printSchema()

root
-- fixed acidity: string (nullable = true)
-- volatile acidity: string (nullable = true)
-- citric acid: string (nullable = true)
-- residual sugar: string (nullable = true)
-- chlorides: string (nullable = true)
-- free sulfur dioxide: string (nullable = true)
-- total sulfur dioxide: string (nullable = true)
-- density: string (nullable = true)
-- pH: string (nullable = true)
-- sulphates: string (nullable = true)
-- alcohol: string (nullable = true)
-- quality: string (nullable = true)

In [0]:
new_data=df.select(*(col(c).cast("float").alias(c) for c in df.columns))
new_data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5.0|
 7.8| 0.88| 0.0| 2.6| 0.098| 25.0| 67.0| 0.9968| 3.2| 0.68| 9.8| 5.0|
 7.8| 0.76| 0.04| 2.3| 0.092| 15.0| 54.0| 0.997|3.26| 0.65| 9.8| 5.0|
 11.2| 0.28| 0.56| 1.9| 0.075| 17.0| 60.0| 0.998|3.16| 0.58| 9.8| 6.0|
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5.0|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
only showing top 5 rows

In [0]:
#Check is there is any missing values are present in the data
null_data=new_data.select([count(when(col(c).isNull(),c)).alias(c) for c in new_data.columns])
null_data.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+

In [0]:
#Assemble all the data
features=new_data.drop("quality")
assemble_data = VectorAssembler(inputCols=features.columns,outputCol="features")
assembler=assemble_data.transform(new_data)
data=assembler.select("features","quality")
data.show(5)

+--------------------+-------+
 features|quality|
+--------------------+-------+
[7.40000009536743...| 5.0|
[7.80000019073486...| 5.0|
[7.80000019073486...| 5.0|
[11.1999998092651...| 6.0|
[7.40000009536743...| 5.0|
+--------------------+-------+
only showing top 5 rows

In [0]:
string=StringIndexer(inputCol="quality",outputCol="Indexquality")
output_string=string.fit(data).transform(data)
output_string.show(5)

+--------------------+-------+------------+
 features|quality|Indexquality|
+--------------------+-------+------------+
[7.40000009536743...| 5.0| 0.0|
[7.80000019073486...| 5.0| 0.0|
[7.80000019073486...| 5.0| 0.0|
[11.1999998092651...| 6.0| 1.0|
[7.40000009536743...| 5.0| 0.0|
+--------------------+-------+------------+
only showing top 5 rows

In [0]:
#Split for training and testing
train,test=output_string.randomSplit([0.7,0.3])

In [0]:
#RandomForestClassifer model
rf=RandomForestClassifier(labelCol="Indexquality",featuresCol="features",numTrees=50)
model=rf.fit(train)

In [0]:
#Get the predictions
prediction=model.transform(test)
prediction.select("Indexquality","prediction","probability").show(5)

+------------+----------+--------------------+
Indexquality|prediction| probability|
+------------+----------+--------------------+
 3.0| 1.0|[0.23760083573483...|
 2.0| 1.0|[0.10955501377934...|
 4.0| 1.0|[0.12812055718119...|
 1.0| 1.0|[0.19481820446140...|
 3.0| 0.0|[0.52490595935161...|
+------------+----------+--------------------+
only showing top 5 rows

In [0]:
#Performance metrics on the test data
evaluation=MulticlassClassificationEvaluator(labelCol="Indexquality",predictionCol="prediction",metricName="accuracy")
prediction_text=evaluation.evaluate(prediction)
print('Accuracy Score is : %s' % prediction_text)

Accuracy Score is : 0.5587583148558758